Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [ ]:
# if you already have a workspace created, just run this to initialize workspace config
# if you haven't created a workspace, skip this
from azureml.core import Workspace

ws = Workspace.from_config()

In case if you do not have, any you can either create it on Azure portal or programmatically:

In [1]:
# if you don't have a workspace already created then run this step
# if workspace already created, skip this step
from azureml.core import Workspace
ws = Workspace.create(name = "cheng_offline_dashboard",
                      subscription_id = "9a2d7383-3c7d-492c-94fc-ba65be408672",
                      resource_group = "cps-dev-cheng",
                      create_resource_group = False,
                      location = 'eastus'
                     )
ws.write_config()
ws.get_details()

Deploying AppInsights with name chengoffinsightsqiqbtigj.
Deployed AppInsights with name chengoffinsightsqiqbtigj.
Deploying ContainerRegistry with name chengoffacrniwgyaxl.
Deployed ContainerRegistry with name chengoffacrniwgyaxl.
Deploying KeyVault with name chengoffkeyvaulttwwvedae.
Deploying StorageAccount with name chengoffstoragexetzcrsu.
Deployed KeyVault with name chengoffkeyvaulttwwvedae.
Deployed StorageAccount with name chengoffstoragexetzcrsu.
Deploying Workspace with name cheng_offline_dashboard.
Deployed Workspace with name cheng_offline_dashboard.
Wrote the config file config.json to: C:\dev\reinforcement_learning\batch\aml_config\config.json


{'id': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourceGroups/cps-dev-cheng/providers/Microsoft.MachineLearningServices/workspaces/cheng_offline_dashboard',
 'name': 'cheng_offline_dashboard',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '51b349c5-5c00-47f4-a69c-2ee18e3ff298',
 'description': '',
 'friendlyName': 'cheng_offline_dashboard',
 'creationTime': '2019-04-01T16:20:28.0067625+00:00',
 'containerRegistry': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.containerregistry/registries/chengoffacrniwgyaxl',
 'keyVault': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.keyvault/vaults/chengoffkeyvaulttwwvedae',
 'applicationInsights': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.insights/components/chengoffinsightsqiqbtigj',
 'identityPrincipalId': 'ddd14bbf-

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
ws.datastores # list out all datastores, this step is optional

In [2]:
import application
from application import context

# set up where to read the log files
ctx = context.Context(
    accountName = 'chengmystorage',
    accountKey = 'OOvXDYUpXqfd9kyipcGyThuVQqh+avvHl36hRcBjeA53FBBtLkx0L0dRJhqpMJV4dXVfYGOWngH9qols20rzPQ==',
    appId = 'large',
    appFolder = 'folder'
)

Now you can create batch training pipeline for vw batch training:

Created pipeline can be either submitted for execution:

Or published to be available via REST endpoint:

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [3]:
from pipelines import extract_pipeline
extractPipeline = extract_pipeline.create_pipeline(ws, ctx, parallel_jobs = 2) # I only parallel running 2 jobs for now for debugging purpose

Creating new AmlCompute cluster: python
Creating
Succeeded............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
Best command selection step is created successfully
Registering datastore for {chengmystoragelarge}
Creating new AmlCompute cluster: extractor
Creating
Succeeded...............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
Cache step is successfully created
Creating new AmlCompute cluster: vw
Creating
Succeeded...............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
Found existing cluster: python
Vw sweep step is successfully created
$AZUREML_DATAREFERENCE_command
Creating new AmlCompute cluster: vw-predict
Creating
Succeeded............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
Vw predict step is successfully created
extractPipeline is succesfully created.
Step Pre

In [4]:
# pipeline params set up the date range of logs

from azureml.core import Experiment  
regen_outputs = False
extractExperiment = Experiment(ws, 'test_sweep').submit(
    extractPipeline, 
    pipeline_params = {
        "start_date":'10/13/2018',
        "end_date":'10/13/2018'
    }, 
    regenerate_outputs = regen_outputs
)
print("Extract Pipeline is submitted for execution")
from azureml.widgets import RunDetails
RunDetails(extractExperiment).show()

Created step Prepare command [eda16ab9][1a2c417c-2a6e-49ce-a936-52aecb8ba997], (This step will run and generate new outputs)
Created step Cache [8b0cd543][78c197e2-a1cc-48ab-9cf0-d2e83b143814], (This step will run and generate new outputs)
Created step Predict [de0e4fec][e7d15e31-311f-4716-ab8e-db190edf92db], (This step will run and generate new outputs)
Created step Best command [6ad4ea8b][bc4e3fca-cf2f-44bf-9b98-1fa939360feb], (This step will run and generate new outputs)
Created step Sweep [vw] [4c9ddfa2][9ae1fe9f-9368-4832-8e86-77887086e6c8], (This step will run and generate new outputs)
Created data reference Application_logs for StepId [1000c488][2dd58e67-bc3a-4b0f-b8c2-126ee8484f4f], (Consumers of this data will generate new runs.)
Submitted pipeline run: 221973dd-80ef-4ad7-b96e-01f2bddb39d5
Extract Pipeline is submitted for execution


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …